In [ ]:
#library imports
import cv2
import numpy as np
import tensorflow as tf
import time
import csv
import os
import shutil
from scipy.ndimage import gaussian_filter
from scipy.spatial import distance
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D

debug_flag = True # use this while tuning the KEY PARAMETERS

filename = "SM2p2_comp" # filename of video to be processed
vid_ext = ".mp4" # extension of video to be processed
# choose from SM3, SM2, SM1.1, SM1.2-4, RR2_low, RR2_mid, RR1 for correct settings
id_str = "SM2"

IMG_SIZE = 64 # needs to be the same as the tf model trained
frame_start = 0 # normally should be 0
# how many frames from frame start will it last for, a really large number just means let the video run until its finished.
frame_end = 1000000000

# for smoothing the tracks before rectification
smoothing_val_small = 2 # this is for horizontal smoothing
smoothing_val = 4 # this is for vertical smoothing
min_ride_duration = 2 # seconds
min_ride_length = 4 # metres

"""%%%%%%%%%%%%%%%%%%% KEY PARAMETERS %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""

########### Background Subtraction Variables #######################
processing_area_bottom = 0.55 # only process breaking waves above this region
processing_area_top = 0.15 # only process breaking waves below this region
processing_area_left = 0.1 # only process breaking waves to the right of this region
processing_area_right = 0.8 # only process breaking waves to the left of this region
# this value was found to be useful to allow more or less motion to be detected for blob detection.
# The value ranges from 0 to 255 and must be an int. The range of 1-50 works best. 0,1 or 2 often 
# does not work due to pixelated nature of videos the higher the motion sensitivity value the better 
# in terms of algorithm speed
motion_sensitivity = 15 
# use (10,10) normally, reduces the sensitivity of motion detection by bluring high movement noisy areas 
# like the foam in the water. 
blur_pixels = (20,20)

# determines the minimum size of the movement seen, this parameter will have some relation to the swell size
blob_area_min = 1 
# determines how many frames back the algorithm looks to compare the current frame with for motion detection
frame_comparison_range = 2 

########### Classification Variables #######################
# determines how confident the algorithm has to be of an object being a breaking wave
classification_threshold = 0.95
# this value determines the size of the snapshots of the detected areas of motion.
# The size of the bounding box will be 'bbox_size'*3 x 'bbox_size'*2 (width x height)
# These snapshots will be sent through a classifier to determine whether they contain a breaking wave or not
# the greater the size of the bouding box the more computations it will take to perform classification, however the
# bounding box needs to be large enough to fit the largest breaking wave inside. Ideally a dynamic bounding box sizer would be ideal
bbox_size = 50

########### Tracking Variables #######################
# need to have tracked an object for at least this many frames for it to be given an ID. 
# Mostly for user visualization purposes. all data is still stored
trackable_threshold = 5 
# 0.5 means height and width are equally weighted for euclidean distance calc (which dist_coef is concerned with).
# 0.9 for example allows more vertical variation but restricts horizontal variation. vice versa for 0.1
LW_ratio = 0.25 
# a larger number makes the algorithm less effective at tracking fast objects, but more reliable in terms of not 
# ID swapping with a neaby object
dist_coef = 70 

expiry_countdown = 3*10 # determines how many frames an object is remembered for while occlusion is occuring
cost_threshold = 1.0 # how lenient is the program in associating a tracked object with an object in the image
time_coef = 1.0 # larger number allows less re-detections, lower number allows more re-detections after a FN or occlusion

# overlap_coef not used, diff_coef not used
# this value is large such that if there is any overlap between an object and the next position of the object, 
# the algorithm considers it the same object
overlap_coef = 1000 
# larger number allows less shape changing of object, lower number allows the shape of the object to 
# change more and still be tracked
diff_coef = 0.6 

cap = cv2.VideoCapture(filename + vid_ext) # put the movie type extension here
frame_rate = cap.get(cv2.CAP_PROP_FPS) #fps

# create a new folder to store all the results in
current_path = os.getcwd()
results_dir = filename + "_results"
results_dir_path = os.path.join(current_path, results_dir)
if os.path.exists(results_dir_path):
    shutil.rmtree(results_dir_path)
    print("old results directory removed")
os.mkdir(results_dir_path)
print("new results directory created called " + results_dir)

cap.set(1, frame_start)

# view the image
ret, frame_rec = cap.read()

frame_skip = 0 # allows for same frame rate for 50fps SM videos
if id_str == "SM3":
    frame_skip = 2
    model_scale_factor_dist = 32
    model_name = 'wave_tracking_tf_lab_sm_v35.model'
    
    #              W1_9, W1_8, W1_7, W2_5, W2_4, W2_0,  C_0, E2_0, E2_2, E2_4, E1_5, E1_7, E1_9,  C_9,  C_8,  C_7,  C_3,  C_4 
    x_vals_pre =  [1653, 1553, 1482, 1832, 1766, 1522,  961,  350,  227,   59,  452,  339,  126,  913,  930,  944,  959,  954] 
    y_vals_pre =  [ 573,  502,  441,  359,  335,  240,  242,  243,  283,  338,  367,  442,  568,  571,  501,  445,  307,  334]
    x_vals_post = [ 187,  215,  246,  301,  329,  478,  478,  478,  402,  330,  302,  246,  187,  187,  215,  245,  365,  330] 
    y_vals_post = [ 234,  234,  234,  164,  164,  164,  303,  454,  455,  455,  380,  379,  379,  303,  303,  303,  304,  304]
    y_vals_post = (606 - np.array(y_vals_post)).tolist()
    easting_scale = 0.040746497
    northing_scale = 0.024422442
    post_rect_name = "IMG_TSBMound3ContourPlot_V000_20220505.png"
    
    if filename[-1] == "5" or filename[-1] == "4":
        test_duration = 320 # seconds
    else:
        test_duration = 55 # seconds
        
elif id_str == "SM2":
    frame_skip = 2
    model_scale_factor_dist = 32
    model_name = 'wave_tracking_tf_lab_sm_v35.model'
    
    # SM2          W1_9, W2_7, W2_4, W3_2, W1_2,  C_2, E1_2, E3_2, E2_4, E2_7, E1_8, E1_9,  C_8, CR_3, CR_2, CR_1
    x_vals_pre =  [1480, 1653, 1424, 1570, 1078,  850,  599,   82,  252,   76,  481,  441,  959,  578,  903, 1102] 
    y_vals_pre =  [ 437,  344,  275,  238,  237,  235,  234,  232,  270,  348,  391,  442,  388,  341,  303,  279]
    x_vals_post = [ 188,  248,  331,  404,  404,  404,  405,  405,  333,  248,  216,  189,  216,  251,  290,  322]
    y_vals_post = [ 243,  171,  171,   82,  244,  316,  395,  550,  474,  474,  395,  395,  316,  383,  315,  260]
    y_vals_post = (631 - np.array(y_vals_post)).tolist()
    easting_scale = 0.040465823
    northing_scale = 0.023454834
    post_rect_name = "IMG_TSBMound2ContourPlot_V000_20220505.png"
    
    if filename[-1] == "5" or filename[-1] == "4":
        test_duration = 320 # seconds
    else:
        test_duration = 55 # seconds
    
elif id_str == "SM1.1":
    frame_skip = 2
    model_scale_factor_dist = 32
    model_name = 'wave_tracking_tf_lab_sm_v35.model'
    
    # SM1.1            1    2    3    4     5     6     7     8    9
    x_vals_pre =  [  197, 171, 318, 760, 1226, 1680, 1744, 1647, 866] 
    y_vals_pre =  [  403, 317, 253, 240,  229,  230,  310,  447, 466]
    x_vals_post = [  131, 169, 217, 228,  240,  235,  173,  119, 114]
    y_vals_post = [  389, 418, 423, 341,  246,  153,  184,  230, 311]
    y_vals_post = (613 - np.array(y_vals_post)).tolist()
    easting_scale = 0.04167927
    northing_scale = 0.024143556
    post_rect_name = "IMG_TSBMound1ContourPlot_V000_20220505.png"
    
    test_duration = 55 # seconds
    
elif id_str == "SM1.2-4":
    frame_skip = 2
    model_scale_factor_dist = 32
    model_name = 'wave_tracking_tf_lab_sm_v35.model'
    
    # SM1.2-4         10,  11,   12,   13,   14
    x_vals_pre =  [  167, 512,  934, 1457, 1542] 
    y_vals_pre =  [  320, 134,  125,  136,  369]
    x_vals_post = [  169, 437,  476,  422,  144]
    y_vals_post = [  420, 474,  313,  119,  229]
    y_vals_post = (613 - np.array(y_vals_post)).tolist()
    easting_scale = 0.04167927
    northing_scale = 0.024143556
    post_rect_name = "IMG_TSBMound1ContourPlot_V000_20220505.png"
    
    if filename[-1] == "4" or filename[-1] == "3":
        test_duration = 320 # seconds
    else:
        test_duration = 55 # seconds
    
elif id_str == "RR2_low":
    model_scale_factor_dist = 29.5
    model_name = 'wave_tracking_tf_lab_sm_v28.model'
    
    #             # G19   G24   G38  G39   G31 G40E  G26  G11
    x_vals_pre =  [ 133, 1209, 1046, 716, 1112, 358, 242, 776] 
    y_vals_pre =  [ 457,  319,  205, 213,  254, 223, 346, 633]
    y_vals_post = [ 392,  237,  235, 311,  236, 394, 393, 316]
    x_vals_post = [ 545,  457,  281, 280,  369, 280, 457, 633]
    x_vals_post = (764 - np.array(x_vals_post)).tolist()
    y_vals_post = (615 - np.array(y_vals_post)).tolist()
    easting_scale = 0.02617801 # design I
    northing_scale = 0.022764228
    post_rect_name = "IMG_DesignIContourPlot_V000_20220505.png"
    
    if filename[-2] == "3":
        test_duration = 329 # seconds
    else:
        test_duration = 50 # seconds
    
elif id_str == "RR2_mid":
    model_scale_factor_dist = 29.5
    model_name = 'wave_tracking_tf_lab_sm_v28.model'
    
    #             # G19   G24   G38  G39   G31 G40E  G26  G11
    x_vals_pre =  [  75, 1225, 1055, 701, 1126, 319, 193, 763] 
    y_vals_pre =  [ 462,  320,  197, 206,  248, 216, 344, 653]
    y_vals_post = [ 392,  237,  235, 311,  236, 394, 393, 316]
    x_vals_post = [ 545,  457,  281, 280,  369, 280, 457, 633]
    x_vals_post = (764 - np.array(x_vals_post)).tolist()
    y_vals_post = (615 - np.array(y_vals_post)).tolist()
    easting_scale = 0.02617801 # design I
    northing_scale = 0.022764228   
    post_rect_name = "IMG_DesignIContourPlot_V000_20220505.png"
    
    if filename[-2:] == "16":
        test_duration = 155 # seconds
    elif filename[-2:] == "14" or filename[-2:] == "15":
        test_duration = 329 # seconds
    else:
        test_duration = 50 # seconds
        
elif id_str == "RR1":
    model_scale_factor_dist = 29.5
    model_name = 'wave_tracking_tf_lab_sm_v24.model'
    
    #             # G18   G24  G38 G40E  G26
    x_vals_pre =  [ 693, 1040, 802, 118, 100] 
    y_vals_pre =  [ 369,  292, 234, 256, 330]
    y_vals_post = [ 295,  222, 220, 369, 368]
    x_vals_post = [ 510,  427, 263, 261, 427]
    x_vals_post = (715 - np.array(x_vals_post)).tolist()
    y_vals_post = (575 - np.array(y_vals_post)).tolist()
    easting_scale = 0.027972028 # design H
    northing_scale = 0.024347826
    post_rect_name = "IMG_DesignHContourPlot_V000_20220505.png"
    test_duration = 329 # seconds
    
    # verification RR1p5_pt1_ground_truth
    #              # G24  G38 G40E  G26
    #x_vals_pre =  [ 349, 869, 845, 317] 
    #y_vals_pre =  [ 646, 623,  84, 115]
    #y_vals_post = [ 222, 220, 369, 368]
    #x_vals_post = [ 427, 263, 261, 427]
    
else:
    print("id_str chosen is not an option, please choose the correct id_str")
    quit()

new_model = tf.keras.models.load_model(model_name)
model_scale_factor_time = model_scale_factor_dist**0.5
total_time = test_duration*model_scale_factor_time
    
####### STEP 1: Find 4 rectification points ######
cv2.imwrite(filename + ".jpg", frame_rec)
pre_rect_name = filename + ".jpg"
pre_rect_image = cv2.imread(pre_rect_name)
post_rect_image = cv2.imread(post_rect_name)
graph_image = post_rect_image.copy()
graph_image = cv2.cvtColor(graph_image, cv2.COLOR_BGR2RGB)

# Read source image.
im_src = cv2.imread(pre_rect_name)

# Group camera coordinates and group map coordinates 
pts_src = []
pts_dst = []
for i in range(0, len(x_vals_pre), 1):
    pts_src.append([x_vals_pre[i], y_vals_pre[i]])
    pts_dst.append([x_vals_post[i], y_vals_post[i]])

pts_src = np.array(pts_src)
pts_dst = np.array(pts_dst)
    
# Read destination image.
im_dst = cv2.imread(post_rect_name)

# Calculate Homography
h, status = cv2.findHomography(pts_src, pts_dst)

# Warp camera image to GPS based on homography
im_out = cv2.warpPerspective(im_src, h, (im_dst.shape[1], im_dst.shape[0]))

# plot points on images
for i in range(0, len(x_vals_pre), 1):
    cv2.circle(im_src, (int(x_vals_pre[i]), int(y_vals_pre[i])), radius=5, 
               color=[0, (1 - i/len(x_vals_pre))*255, (i/len(x_vals_pre))*255], thickness=-1)
    cv2.circle(im_dst, (int(x_vals_post[i]), int(y_vals_post[i])), radius=3, 
               color=[0, (1 - i/len(x_vals_pre))*255, (i/len(x_vals_pre))*255], thickness=-1)

# Display images
cv2.namedWindow("Source Image", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Source Image", 1280, 720)
cv2.imshow("Source Image", im_src)
cv2.imshow("Destination Image", im_dst)
cv2.imshow("Warped Source Image", im_out)
cv2.waitKey()
cv2.destroyAllWindows()

print("done")

In [ ]:
"""
This function does all the image preperation before classification and tracking. The basic process
involves reducing noise before motion detection (blurring frames to be compared), finding the difference between
frames (motion detection) and then blob detection to segment the most significant motion (blob detection).

Outputs: keypoints (special 'pt' type)
"""
def motion_detection(frame, comparison_frame, blur_pixels, blob_params, motion_sensitivity, bbox_size):
    # blur frame before taking diff image
    blur = cv2.blur(frame, blur_pixels)
    blur_comparison = cv2.blur(comparison_frame, blur_pixels)

    # motion detection
    frame_diff_blur = cv2.subtract(blur, blur_comparison)

    # blob detection process
    detector = cv2.SimpleBlobDetector_create(blob_params)
    frame_diff_blur_gray = cv2.cvtColor(frame_diff_blur, cv2.COLOR_BGR2GRAY)
    _, frame_diff_blur_BW = cv2.threshold(frame_diff_blur_gray, motion_sensitivity, 255, cv2.THRESH_BINARY)
    frame_diff_blur_BW = cv2.bitwise_not(frame_diff_blur_BW)

    blobs_remaining = 100 # arbitrary number larger than 1 to start the loop
    first_pass_blob = True
    while blobs_remaining > 0:
        if first_pass_blob == True:
            keypoints_i = detector.detect(frame_diff_blur_BW)
            keypoints = list(keypoints_i)
            first_pass_blob = False
        else:
            keypoints_i = detector.detect(frame_diff_blur_BW)
            if len(keypoints_i) == 0:
                pass
            else:
                keypoints.extend(keypoints_i)
        
        # mask out each detected keypoint
        for keypoint_i in keypoints_i:
            x_start_rec = int(keypoint_i.pt[0] - (3 / 2 * bbox_size))
            x_end_rec = int(keypoint_i.pt[0] + (3 / 2 * bbox_size))
            y_start_rec = int(keypoint_i.pt[1] - (1 * bbox_size))
            y_end_rec = int(keypoint_i.pt[1] + (1 * bbox_size))
            start_point_rec = (x_start_rec, y_start_rec)
            end_point_rec = (x_end_rec, y_end_rec)
            cv2.rectangle(frame_diff_blur_BW, start_point_rec, end_point_rec, color=(255, 255, 255), thickness=-1)
        
        kernel = np.ones((int(bbox_size/5), int(bbox_size/5)), np.uint8)
        frame_diff_blur_BW = cv2.morphologyEx(frame_diff_blur_BW, cv2.MORPH_CLOSE, kernel)
        
        blobs_remaining = len(keypoints_i)
        #print("blobs remaining = " + str(blobs_remaining))
        
    for i in range(0, len(keypoints), 1):
        cv2.circle(frame_diff_blur_BW, (int(keypoints[i].pt[0]), int(keypoints[i].pt[1])), 
                   radius=5, color=[150, 150, 150], thickness=-1)
    
    cv2.namedWindow("blobs", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("blobs", 1280, 720)
    cv2.imshow("blobs", frame_diff_blur_BW)

    return keypoints

"""
This function takes snapshots (of size of the bounding boxes) of all the detected areas of motion. Then it classifies
each of the snapshots as a breaking wave or not a breaking wave. The relevant data associated with each breaking wave is
stored in the 'objects' list. 

Outputs: wave_detected (bool)
"""
def breaking_wave_classification(centre_frame, keypoints, bbox_size, classification_threshold, frame_count, 
                                 expiry_countdown, processing_area_bottom, processing_area_top, processing_area_left, 
                                 processing_area_right, debug_flag, frame_show):
    global objects
    global IMG_SIZE

    objects = [] #initialize this list variable

    IMG_SIZE_WID = int(IMG_SIZE) # can change this if needed
    image_data = []
    tf_images = []
    for keyPoint in keypoints:
        # crop an image for each blob (located within the bounding box).
        x_start = int(keyPoint.pt[0] - (3 / 2 * bbox_size))
        x_finish = int(keyPoint.pt[0] + (3 / 2 * bbox_size))
        y_start = int(keyPoint.pt[1] - (1 * bbox_size))
        y_finish = int(keyPoint.pt[1] + (1 * bbox_size))

        if y_finish > processing_area_bottom*centre_frame.shape[0]:
            continue # don't process this blob
        if y_start < processing_area_top*centre_frame.shape[0]:
            continue # don't process this blob
        if x_start < processing_area_left*centre_frame.shape[1]:
            continue # don't process this blob
        if x_finish > processing_area_right*centre_frame.shape[1]:
            continue # don't process this blob


        # error handling for the boundary of the image
        if x_start <= 0:
            continue
        elif x_finish >= centre_frame.shape[1]:
            continue
        elif y_start <= 0:
            continue
        elif y_finish >= centre_frame.shape[0]:
            continue
        else:  # crop the image
            image_data.append([int(keyPoint.pt[0]), int(keyPoint.pt[1])])
            colour_image = centre_frame[y_start: y_finish, x_start: x_finish]
            resized_image = cv2.resize(colour_image, (IMG_SIZE_WID, IMG_SIZE))
            tf_images.append(resized_image)
        
        if debug_flag == True:
            cv2.circle(frame_show, (int(keyPoint.pt[0]), int(keyPoint.pt[1])), radius=5, color=[100, 100, 100], thickness=-1)
        
    if len(tf_images) < 1:
        wave_detected = False
    else:
        wave_detected = False
        image_data_counter = 0
        imshow_tf_images = tf_images
        tf_images = np.array(tf_images)
        tf_images = tf.keras.utils.normalize(tf_images, axis=1)
        tf_images = tf_images.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)
        predictions = new_model.predict(tf_images)
        for prediction in predictions:
            #print(prediction[0])
            #cv2.imshow("thresh_image", imshow_tf_images[image_data_counter])
            #cv2.waitKey()
            if prediction[0] > classification_threshold:
                #print(prediction[0])
                #cv2.imshow("thresh_image", imshow_tf_images[image_data_counter])
                #cv2.waitKey()

                # add all relevant values to 'objects' list
                x_centroid = image_data[image_data_counter][0]
                y_centroid = image_data[image_data_counter][1]
                # both need to be lists since more centroid coordinates will be appended when tracked
                centroid_x = [x_centroid]
                centroid_y = [y_centroid]
                detection_timestamp = [frame_count]

                blob_cloud_mask = tf_images[image_data_counter]
                # blob_cloud_mask functionality not added yet

                object_contents = [centroid_x, centroid_y, blob_cloud_mask, detection_timestamp, expiry_countdown, 0, False]
                objects.append(object_contents)

                # flag that a wave has been detected
                wave_detected = True
                
                if debug_flag == True:
                    cv2.circle(frame_show, (x_centroid, y_centroid), radius=5, color=[0, 255, 0], thickness=-1)
            else:
                pass
                #print(prediction[0])
                #cv2.imshow("thresh_image", imshow_tf_images[image_data_counter])
                #cv2.waitKey()
            image_data_counter = image_data_counter + 1

    return wave_detected, frame_show

"""
This function updates the status of a tracked object. Outputs a boolean, 
which is true if the previous timestamp was the same for the object. False otherwise.

Outputs: bool
"""
def update_tracked_object(object, expiry_countdown, closest_tracked_object_index):

    global tracked_objects

    # if the previous timestamp is the same, take the average of the timestamp and 
    # set the new centroid to the average of the two centroids.
    if tracked_objects[closest_tracked_object_index][3][-1] == object[3][0]:
        # add centroids to tracked object
        tracked_objects[closest_tracked_object_index][0][-1] = int((object[0][0] + 
                                                                    tracked_objects[closest_tracked_object_index][0][-1])/2)
        tracked_objects[closest_tracked_object_index][1][-1] = int((object[1][0] + 
                                                                    tracked_objects[closest_tracked_object_index][1][-1])/2)
        # THIS METHOD IS SLIGHTLY SKEWED TOWARDS THE POSITION OF LAST POINT WITH THE SAME TIMESTAMP.
        # MOVING AVERAGE ONLY IS ACCURATE FOR 2 POINTS, HOWEVER STILL DEEMED ACCEPTABLE FOR MORE THAN
        # 2 POINTS IF CLOSELY LOCATED

        # update mask -> simply pick the bigger mask
        # MASK ACTUALLY ISNT DOING ANYTHING - won't do anything until functionality is added later
        if np.sum(object[2]) > np.sum(tracked_objects[closest_tracked_object_index][2]):
            tracked_objects[closest_tracked_object_index][2] = object[2]
        else:
            # don't change the mask
            pass

        # reset expiry count to initial value
        tracked_objects[closest_tracked_object_index][4] = expiry_countdown

        return True
    else:
        # add centroids to tracked object
        tracked_objects[closest_tracked_object_index][0].append(object[0][0])
        tracked_objects[closest_tracked_object_index][1].append(object[1][0])
        # update mask
        tracked_objects[closest_tracked_object_index][2] = object[2]
        # add timestamp to tracked object
        tracked_objects[closest_tracked_object_index][3].append(object[3][0])
        # reset expiry count to initial value
        tracked_objects[closest_tracked_object_index][4] = expiry_countdown

        return False
"""
This function updates the coordinates and timestamps of object tracked in the tracked_objects list.
Metrics to determine if an object is identified in a new position are the following:
-overlapping of blob mask
-centroid distance
-blob_mask area similarity
-time difference between detections

Outputs: graphing_mat (list), frame (opencv image)
"""
def track_blobs(frame, frame_show, objects, expiry_countdown, centroid_norm_param, blob_sim_norm_param, 
                dist_coef, time_coef, diff_coef, overlap_coef, graphing_mat, graph, bbox_size, LW_ratio, 
                post_rect_image, h):
    global tracked_objects
    global cost_threshold

    closest_tracked_object_index = 0 # simply initializing variable to avoid warnings
    # for each object detected, compare to all tracked objects to see if there is enough correlation to associate them.
    for object in objects:
        tracking_index = 0
        object_has_been_tracked = False
        # initialized as a relatively large variable, this value is updated according to the 
        # object association with the lowest cost.
        cost_min = 100 
        for tracked_object in tracked_objects:
            # check blob mask overlapping
            object_mask = object[2]
            tracked_object_mask = tracked_object[2]

            # add cost to the if statement, such that the cost is a combination of the Metrics below
            centroid_dist = ((((((object[0][0] - tracked_object[0][-1])*LW_ratio) ** 2) +
                               ((((object[1][0] - tracked_object[1][-1])*(1-LW_ratio)) ** 2))
                               ) ** 0.5) / centroid_norm_param) * dist_coef
            time_difference = ((object[3][0] - tracked_object[3][-1]) / expiry_countdown) * time_coef
            blob_overlap_value = (np.sum(object_mask * tracked_object_mask)/np.sum(object_mask))*overlap_coef
            #blob_area_difference = ((((int(np.sum(tracked_object_mask)) - 
            #                           int(np.sum(object_mask)))**2)**0.5)/blob_sim_norm_param)*diff_coef
            # overlap acts like a 'free pass' if there is any overlap, since it will negate the build up of the cost
            cost = centroid_dist + time_difference #+ blob_area_difference - blob_overlap_value

            #debugging
            #print("overlap: " + str(blob_overlap_value))
            #print("centroid_distance: " + str(centroid_dist))
            #print("area_difference: " + str(blob_area_difference))
            #print("time_difference: " + str(time_difference))
            #print("tracked object: " + str(tracked_object[3][-1]) + " object: " + str(object[3][0]))
            #print("cost: ", cost)
            #print("\n")
            #cv2.namedWindow('tracked object', cv2.WINDOW_NORMAL)
            #cv2.resizeWindow('tracked object', 480, 270)
            #cv2.imshow("tracked object", tracked_object_mask)
            #cv2.namedWindow('object', cv2.WINDOW_NORMAL)
            #cv2.resizeWindow('object', 480, 270)
            #cv2.imshow("object", object_mask)
            #cv2.waitKey()

            # iterate through all tracked objects and keep the index of the object that is most likely associated
            if cost < cost_threshold:
                if cost < cost_min:
                    cost_min = cost
                    closest_tracked_object_index = tracking_index
                    object_has_been_tracked = True

            tracking_index = tracking_index + 1

        # if object could be associated with an older tracked object, 
        # update the tracked object by adding the associated object data
        if object_has_been_tracked == True:
            same_object = update_tracked_object(object, expiry_countdown, closest_tracked_object_index)

            # If the object is 'trackable' (has been tracked at least 'trackable_threshold' times), 
            # plot currently visible tracked objects
            if tracked_objects[closest_tracked_object_index][6] == True:
                # only plot if its a new object. Don't print objects that have the same 'closest_tracked_object_index'
                if same_object == False:
                    point1 = (int(tracked_objects[closest_tracked_object_index][0][-1] - (3 / 2 * bbox_size)), 
                              int(tracked_objects[closest_tracked_object_index][1][-1] - (1 * bbox_size)))
                    point2 = (int(tracked_objects[closest_tracked_object_index][0][-1] + (3 / 2 * bbox_size)), 
                              int(tracked_objects[closest_tracked_object_index][1][-1] + (1 * bbox_size)))
                    cv2.circle(frame_show, (tracked_objects[closest_tracked_object_index][0][-1], 
                                            tracked_objects[closest_tracked_object_index][1][-1]), 
                               radius=2, color=[100, 255, 255])
                    cv2.rectangle(frame_show, point1, point2, (0, 150, 255), 3)
                    cv2.putText(frame_show, str(tracked_objects[closest_tracked_object_index][5]), 
                                (tracked_objects[closest_tracked_object_index][0][-1], 
                                 tracked_objects[closest_tracked_object_index][1][-1]),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

                    # update the graphing data, including the unique colour for that tracked object
                    # NOTE: initial setting up of graphing_mat is done within tracked_object_dropout() function
                    graphing_mat[tracked_objects[closest_tracked_object_index][5]].append(
                        [(tracked_objects[closest_tracked_object_index][0][-1], 
                          tracked_objects[closest_tracked_object_index][1][-1])])
                    graph_colour_1 = int(((tracked_objects[closest_tracked_object_index][5]) * 100) % 255)
                    graph_colour_2 = int(((tracked_objects[closest_tracked_object_index][5]) * 15) % 255)
                    graph_colour_3 = int((255 - ((tracked_objects[closest_tracked_object_index][5]) * 50)) % 255)
                    # draw on the graph
                    cv2.circle(graph, (tracked_objects[closest_tracked_object_index][0][-1], 
                                       tracked_objects[closest_tracked_object_index][1][-1]), 1, 
                               (graph_colour_1, graph_colour_2, graph_colour_3), 2)

                    # also draw rectified graph
                    graph_p1 = tracked_objects[closest_tracked_object_index][0][-1]
                    graph_p2 = tracked_objects[closest_tracked_object_index][1][-1]
                    print([graph_p1, graph_p2])
                    output = cv2.perspectiveTransform(np.array([[[graph_p1, graph_p2]]], dtype="float32"), h)
                    print(output)
                    cv2.circle(post_rect_image, (int(output[0][0][0]), int(output[0][0][1])), 1,
                               (graph_colour_1, graph_colour_2, graph_colour_3), 2)

        # if the object detected could not be linked to any previous objects, add it as a new object
        else:
            tracked_objects.append(object)

    return graphing_mat, frame, frame_show

"""
Checks if any of the tracked objects have expired. Also update trackable status where
if the object has been tracked at least 'trackable_threshold' times, then it can be given an ID

Outputs: graphing_mat (list)
"""
def tracked_object_dropout(trackable_threshold, graphing_mat, graph, filename, post_rect_image, h):
    global tracked_objects
    global tracking_ID

    surviving_tracked_objects = [] # used to hold on to tracekd objects that haven't expired

    # first subtract 1 expiry countdown value from every tracked object
    for tracked_object in tracked_objects:
        # if object won't expire, subtract 1 from expiry countdown and keep object
        if tracked_object[4] > 1:
            tracked_object[4] = tracked_object[4] - 1
            surviving_tracked_objects.append(tracked_object)

            # update trackable status when acceptable:
            if tracked_object[6] == False:
                # if there have been more than the threshold of timestamps
                if len(tracked_object[3]) > trackable_threshold:
                    # set status as trackable
                    tracked_object[6] = True
                    # give a tracking ID
                    tracked_object[5] = tracking_ID
                    tracking_ID = tracking_ID + 1

                    # add to graphing_mat
                    for i in range(0, trackable_threshold, 1):
                        graphing_mat.append([(tracked_object[0][i], tracked_object[1][i])])
                        # update the graphing data, including the unique colour for that tracked object
                        graph_colour_1 = int(((tracked_object[5]) * 100) % 255)
                        graph_colour_2 = int(((tracked_object[5]) * 15) % 255)
                        graph_colour_3 = int((255 - ((tracked_object[5]) * 50)) % 255)
                        # draw on the graph
                        cv2.circle(graph, (tracked_object[0][i], tracked_object[1][i]), 1,
                                   (graph_colour_1, graph_colour_2, graph_colour_3), 2)

                        # also draw rectified graph
                        graph_p1 = tracked_object[0][i]
                        graph_p2 = tracked_object[1][i]
                        print([graph_p1, graph_p2])
                        output = cv2.perspectiveTransform(np.array([[[graph_p1, graph_p2]]], dtype="float32"), h)
                        print(output)
                        cv2.circle(post_rect_image, (int(output[0][0][0]), int(output[0][0][1])), 1,
                                   (graph_colour_1, graph_colour_2, graph_colour_3), 2)

        # else write object to the data file and don't add to surviving_tracked_objects list
        else:
            sat_x = []
            sat_y = []
            # also draw rectified graph
            graph_p1 = tracked_object[0]
            graph_p2 = tracked_object[1]
            for gp in range(0, len(graph_p1), 1):
                output_2d = cv2.perspectiveTransform(np.array([[[graph_p1[gp], graph_p2[gp]]]], dtype="float32"), h)
                sat_x.append(output_2d[0][0][0])
                sat_y.append(output_2d[0][0][1])
            print(np.array([tracked_object[0], tracked_object[1], tracked_object[3], sat_x, sat_y]))
            # Data Format: [x_centroid coordinate, y_centroid coordinate, timestamps (frames), 
            #               sat_x_coordinate, sat_y_coordinate]
            data_csv = np.array([tracked_object[0], tracked_object[1], tracked_object[3], sat_x, sat_y])
            # 1 if tracked for more than 'trackable_threshold' frames, 0 if tracked for less
            space_csv = np.array([int(tracked_object[6])])
            # write 5 rows of data, x coordinates, y coordinates and frame and rectified coordinates
            with open(results_dir + "/tracked_objects" + filename + ".csv", "ab") as csv_file:
                np.savetxt(csv_file, data_csv, delimiter=",", fmt='%f')
                np.savetxt(csv_file, space_csv, delimiter=",", fmt='%f')

    # re-create 'tracked_objects' from the surviving objects
    tracked_objects = surviving_tracked_objects

    return graphing_mat


"""%%%%%%%%%%%%%%%%%%% FRAME COMPARISON SETUP %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# These frames are used to allow motion detection by taking a difference image between different frames in the while loop.
# The difference image will be between the current frame in the video 'frame' and the last frame 'comparison_frame'. 
# By changing the frames between the current frame and comparison frame more or less motion can be detected.
frame_list = []
for i in range(0, frame_comparison_range, 1):
    _, framei = cap.read()
    frame_list.append(framei)

comparison_frame = frame_list[(frame_comparison_range - 1)]
centre_frame = frame_list[int(frame_comparison_range/2)]

"""%%%%%%%%%%%%%%%%%%% INITIALIZATIONS %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# general initialisations
centroid_norm_param = 0
blob_similarity_norm_param = 0
tracking_ID = 0
objects = []
tracked_objects = []

# initialize file for saving data in
open(results_dir + "/tracked_objects" + filename + ".csv", "wb")
with open(results_dir + "/tracked_objects" + filename + ".csv", "a") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["homography matrix"])
with open(results_dir + "/tracked_objects" + filename + ".csv", "ab") as csv_file:
    np.savetxt(csv_file, h, delimiter=",")
with open(results_dir + "/tracked_objects" + filename + ".csv", "a") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["data"])

# blob parameter setup
blob_params = cv2.SimpleBlobDetector_Params()
blob_params.filterByArea = False # True
blob_params.minArea = blob_area_min # how many pixels is acceptable per blob
blob_params.filterByCircularity = False # True
blob_params.minCircularity = 0.01
blob_params.filterByConvexity = False # True
blob_params.minConvexity = 0.01
blob_params.filterByInertia = False # True
blob_params.minInertiaRatio = 0.01

##### graphing setup ########
# graphing variables
graphing_mat = []
graph_ID = -1 # start at -1, if image tracking to be graphed it is initialized at 0
# set the graph background to be a frame of the video to give the user a reference point for the tracked waves
graph = frame_list[0] 

# plot boundaries of the region of the image to be processed
cv2.line(graph, (0, int(processing_area_bottom*graph.shape[0])), 
         (graph.shape[1], int(processing_area_bottom*graph.shape[0])), (0,0,255), thickness=3)
cv2.line(graph, (0, int(processing_area_top*graph.shape[0])), 
         (graph.shape[1], int(processing_area_top*graph.shape[0])), (0,0,255), thickness=3)
cv2.line(graph, (int(processing_area_left*graph.shape[1]), 0), 
         (int(processing_area_left*graph.shape[1]), graph.shape[0]), (0,0,255), thickness=3)
cv2.line(graph, (int(processing_area_right*graph.shape[1]), 0), 
         (int(processing_area_right*graph.shape[1]), graph.shape[0]), (0,0,255), thickness=3)

"""%%%%%%%%%%%%%%%%%%% MAIN WHILE LOOP %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# 588 frames in Snapper_Rocks_10_05_2017.mp4, therefore frame rate = 29.97 fps or 30fps
run_frame_rate = 1000 # a little bit faster just because of the time delays associated with the rest of the code.
frame_count = frame_comparison_range # this value is a counter so not 0 indexed
first_pass = True
frame_start_count = 0
frame_end_count = 0
frame_show = cap.read()
cap.set(1, frame_start)
while(cap.isOpened()):

    ret, frame = cap.read()
    if frame_end_count > frame_end:
        break
    
    if frame_skip > 0:
        if frame_end_count % frame_skip == 0:
            frame_count = frame_count + 1 
            frame_end_count = frame_end_count + 1
            continue

    if ret == True:
        frame_show = frame.copy()
        
        # debugging for vidos that dont play at correct frame rate and have duplicate frames
        frame_diff_initial = cv2.subtract(frame, frame_list[0])
        if cv2.sumElems(cv2.sumElems(frame_diff_initial))[0] < 1500000:
            frame_count = frame_count + 1 
            frame_end_count = frame_end_count + 1
            continue
        else:
            """%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% STEP 1: MOTION DETECTION %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
            # track motion in the image
            keypoints = motion_detection(frame, comparison_frame, blur_pixels, blob_params, motion_sensitivity, bbox_size)

            """%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% STEP 2: BREAKING WAVE CLASSIFICATION %%%%%%%%%%%%%%%%%%"""
            # classify areas of motion as breaking waves or not breaking waves. Keep those classfied as breaking waves
            # follow through with tracking if any waves have been detected
            classify_check, frame_show = breaking_wave_classification(centre_frame, keypoints, bbox_size, 
                                                                      classification_threshold, frame_count, 
                                                                      expiry_countdown, processing_area_bottom, 
                                                                      processing_area_top, processing_area_left, 
                                                                      processing_area_right, debug_flag, frame_show)
            if classify_check == True:

                if first_pass == True:
                    # set objects as first tracked objects
                    tracked_objects.extend(objects)
                    first_pass = False

                    # set normalization parameters
                    centroid_norm_param = int((frame.shape[0] ** 2 + frame.shape[1] ** 2) ** 0.5)
                    blob_similarity_norm_param = int(frame.shape[0] * frame.shape[1])
                    
                else:
                    # track the waves
                    graphing_mat, frame, frame_show = track_blobs(frame, frame_show, objects, expiry_countdown, 
                                                                  centroid_norm_param, blob_similarity_norm_param, 
                                                                  dist_coef, time_coef, diff_coef, overlap_coef, 
                                                                  graphing_mat, graph, bbox_size, LW_ratio, 
                                                                  post_rect_image, h)
                    # check for dropout in tracking process
                    graphing_mat = tracked_object_dropout(trackable_threshold, graphing_mat, graph, filename, 
                                                          post_rect_image, h)

            """%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% STEP 4: PLOTTING %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
            cv2.namedWindow("graph", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("graph", 1280, 720)
            cv2.imshow("graph", graph)
            cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("frame", 1280, 720)
            cv2.imshow("frame", frame)
            cv2.namedWindow("frame_vid", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("frame_vid", 1280, 720)
            cv2.imshow("frame_vid", frame_show)
            cv2.imshow("rect_graph", post_rect_image)
            cv2.waitKey(int(1000/run_frame_rate)) # essentially just means run the video as fast as possible

            # frame comparison, cycle through the list so only 'frame_comparison_range' frames are stored
            frame_list.insert(0, frame)
            frame_list.pop(frame_comparison_range)
            comparison_frame = frame_list[(frame_comparison_range - 1)]
            centre_frame = frame_list[int(frame_comparison_range / 2)]
            
        frame_count = frame_count + 1 
        frame_end_count = frame_end_count + 1
        print(frame_end_count)
    else:
        break

# Expire all wave tracks
for i in range(0, expiry_countdown, 1):
    graphing_mat = tracked_object_dropout(trackable_threshold, graphing_mat, graph, filename, post_rect_image, h)

# initialize file for saving data in
open(results_dir + "/" + "tracking_settings_" + filename + ".csv", "wb")
with open(results_dir + "/" + "tracking_settings_" + filename + ".csv", "a") as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(["frame_rate_vid", frame_rate, "frame_count", frame_end_count,
                     "wave_tracking_model", model_name,
                     "IMG_SIZE", IMG_SIZE, "processing_area_bottom", processing_area_bottom, 
                     "processing_area_top", processing_area_top, "processing_area_left", processing_area_left, 
                     "processing_area_right", processing_area_right, 
                     "classification_threshold", classification_threshold, "frame_comparison_range",
                     frame_comparison_range, "blob_area_min", blob_area_min,
                     "motion_sensitivity", motion_sensitivity, "trackable_threshold", trackable_threshold,
                     "expiry_countdown", expiry_countdown, "cost_threshold", cost_threshold,
                     "dist_coef", dist_coef, "LW_ratio", LW_ratio,
                     "time_coef", time_coef, "bbox_size", bbox_size, "blur_pixels", blur_pixels])

# save an image of the graph
cv2.imwrite(results_dir + "/" + filename + "graph.jpg", graph)

# save an image of the graph
cv2.imwrite(results_dir + "/" + filename + "birds_eye.jpg", post_rect_image)

print(str("Tracking finished"))
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Do Gaussian smoothing before rectification

# open csv file
# first load the csv file into python with rectified values
csv_file_name = results_dir + "/tracked_objects" + filename + ".csv"
# get all the tracks before rectification
all_u_i = []
all_v_i = []
all_time_i = []
with open(csv_file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    wave_tracked = []
    all_u = []
    all_v = []
    all_time = []
    for row in csv_reader:
        if line_count < 7:
            # not a data cell
            pass
        # if at the end of a wave track
        elif (line_count - 7) % 6 == 5:
            if '' in row:
                blank = row.index('')
                row = row[0:blank]
            else:
                row = row[0:]
            row = np.array([float(i) for i in row]) # convert strings to floats
            wave_tracked.append(row)
            # if wave lasted long enough
            if float(wave_tracked[3][0]) == 1:
                # append full set of wave track data to list
                all_u_i.append(all_u)
                all_v_i.append(all_v)
                all_time_i.append(all_time)
            else:
                # don't append since wasn't flagged as a tracked wave (didn't last long enough)
                pass
            wave_tracked = []
            all_u = []
            all_v = []
            all_time = []
        else:
            # if u value
            if (line_count - 7) % 6 == 0:
                if '' in row:
                    blank = row.index('')
                    row = row[0:blank]
                else:
                    row = row[0:]
                u_row = np.array([float(i) for i in row]) # convert strings to floats
                wave_tracked.append(u_row)
                all_u.append(u_row)
            # if v value
            elif (line_count - 7) % 6 == 1:
                if '' in row:
                    blank = row.index('')
                    row = row[0:blank]
                else:
                    row = row[0:]
                v_row = np.array([float(i) for i in row]) # convert strings to floats
                wave_tracked.append(v_row)
                all_v.append(v_row)
            elif (line_count - 7) % 6 == 2:
                if '' in row:
                    blank = row.index('')
                    row = row[0:blank]
                else:
                    row = row[0:]
                time_row = np.array([float(i) for i in row]) # convert strings to floats
                wave_tracked.append(time_row)
                all_time.append(time_row)
            else:
                pass

        line_count = line_count + 1

smooth_graph = cv2.imread(pre_rect_name)
        
smooth_u_i = []
smooth_v_i = []
smooth_x_i = []
smooth_y_i = []
for i in range(0, len(all_time_i), 1):
    
    # smooth u and v values before rectification
    smooth_u_i.append(gaussian_filter(all_u_i[i], sigma=smoothing_val_small)) # 1 is very small to avoid track trunctation
    smooth_v_i.append(gaussian_filter(all_v_i[i], sigma=smoothing_val))
    
    # rectify all track points
    smooth_graph_colour_1 = int(((i) * 100) % 255)
    smooth_graph_colour_2 = int(((i) * 15) % 255)
    smooth_graph_colour_3 = int((255 - ((i) * 50)) % 255)
    smooth_xs = []
    smooth_ys = []
    for j in range(0, len(all_time_i[i][0]), 1):
        output_2d = cv2.perspectiveTransform(np.array([[[smooth_u_i[i][0][j], smooth_v_i[i][0][j]]]], dtype="float32"), h)
        smooth_xs.append(output_2d[0][0][0])
        smooth_ys.append(output_2d[0][0][1])
        
        # draw on the graph
        cv2.circle(smooth_graph, (int(smooth_u_i[i][0][j]), int(smooth_v_i[i][0][j])), 1, 
                   (smooth_graph_colour_1, smooth_graph_colour_2, smooth_graph_colour_3), 2)
    
    smooth_x_i.append(smooth_xs)
    smooth_y_i.append(smooth_ys)
    
cv2.namedWindow("smooth_graph", cv2.WINDOW_NORMAL)
cv2.resizeWindow("smooth_graph", 1280, 720)
cv2.imshow('smooth_graph', smooth_graph)
cv2.waitKey()
cv2.destroyAllWindows()

# initialize file for saving data in
open(results_dir + "/tracked_objects_smooth_" + filename + ".csv", "wb")
with open(results_dir + "/tracked_objects_smooth_" + filename + ".csv", "a") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["homography matrix"])
with open(results_dir + "/tracked_objects_smooth_" + filename + ".csv", "ab") as csv_file:
    np.savetxt(csv_file, h, delimiter=",")
with open(results_dir + "/tracked_objects_smooth_" + filename + ".csv", "a") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["data"])

# save as csv
for i in range(0, len(all_time_i), 1):
    data_csv = np.array([smooth_u_i[i][0], smooth_v_i[i][0], all_time_i[i][0], smooth_x_i[i], smooth_y_i[i]])
    # 1 if tracked for more than 'trackable_threshold' frames, 0 if tracked for less
    space_csv = np.array([1])
    # write 5 rows of data, x coordinates, y coordinates and frame and rectified coordinates
    with open(results_dir + "/tracked_objects_smooth_" + filename + ".csv", "ab") as csv_file:
        np.savetxt(csv_file, data_csv, delimiter=",", fmt='%f')
        np.savetxt(csv_file, space_csv, delimiter=",", fmt='%f')

print("Now producing stats and figures")

%matplotlib qt

colours_list = ['#FF0000', '#FFAC00', '#FFFF00', '#B9FF00', '#00FF00']
legend_list = ['< 4s' , '< 8s'  , '< 12s'  , '< 16s'  , '>= 16s' ]

custom_lines = []
for j in range(0, len(legend_list), 1):
    custom_lines.append(Line2D([0], [0], color=colours_list[j], lw=4))

def find_true_north_dir(rise, run):
    
    if rise == 0 and run > 0:
        deg = 90
    elif rise == 0 and run < 0:
        deg = 270
    elif rise > 0 and run == 0:
        deg = 0
    elif rise < 0 and run == 0:
        deg = 180
    elif run > 0:
        deg = 90 - np.degrees(math.atan(rise/run))
    elif run < 0:
        deg = 180 + 90 - np.degrees(math.atan(rise/run))
    else:
        print("unexpected condition for direction")
        quit()
    return deg
    
# first load the csv file into python with rectified values
csv_file_name = results_dir + "/tracked_objects_smooth_" + filename + ".csv"
# these 4 lists are different ways to store data which are useful in different ways for statistics
all_eastings_i = []
all_northings_i = []
all_times_i = []
with open(csv_file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    wave_tracked = []
    all_easting = []
    all_northing = []
    all_time = []
    for row in csv_reader:
        if line_count < 7:
            # not a data cell
            pass
        # if at the end of a wave track
        elif (line_count - 7) % 6 == 5:
            if '' in row:
                blank = row.index('')
                row = row[0:blank]
            else:
                row = row[0:]
            row = np.array([float(i) for i in row]) # convert strings to floats
            wave_tracked.append(row)
            # if wave lasted long enough
            if float(wave_tracked[3][0]) == 1:
                # append full set of wave track data to list
                all_eastings_i.append(all_easting)
                all_northings_i.append(all_northing)
                all_times_i.append(all_time)
            else:
                # don't append since wasn't flagged as a tracked wave (didn't last long enough)
                pass
            wave_tracked = []
            all_easting = []
            all_northing = []
            all_time = []
        else:
            # if x value, convert to easting
            if (line_count - 7) % 6 == 3:
                if '' in row:
                    blank = row.index('')
                    row = row[0:blank]
                else:
                    row = row[0:]
                easting_row = np.array([float(i)*easting_scale for i in row]) # convert strings to floats
                wave_tracked.append(easting_row)
                all_easting.append(easting_row)
            # if y value, convert to northing
            elif (line_count - 7) % 6 == 4:
                if '' in row:
                    blank = row.index('')
                    row = row[0:blank]
                else:
                    row = row[0:]
                northing_row = np.array([float(i)*-northing_scale for i in row]) # convert strings to floats
                wave_tracked.append(northing_row)
                all_northing.append(northing_row)               
            elif (line_count - 7) % 6 == 2:
                if '' in row:
                    blank = row.index('')
                    row = row[0:blank]
                else:
                    row = row[0:]
                time_row = np.array([float(i) for i in row]) # convert strings to floats
                wave_tracked.append(time_row)
                all_time.append(time_row)
            else:
                pass
                
        line_count = line_count + 1
    
all_eastings = []
all_northings = []
all_times = []
# remove any tracks outside image
for i in range(0, len(all_eastings_i), 1):
    
    # check duration of each track
    if ((max(all_times_i[i][0]) - min(all_times_i[i][0])) / frame_rate)*model_scale_factor_time < min_ride_duration:
        continue

    # approximate ride length of waves - simply measure start to finish point
    smooth_easting_i = np.array(all_eastings_i[i])*model_scale_factor_dist
    smooth_northing_i = np.array(all_northings_i[i])*model_scale_factor_dist

    start_point = [smooth_easting_i[0][0], smooth_northing_i[0][0]]
    finish_point = [smooth_easting_i[0][-1], smooth_northing_i[0][-1]]
    ride_length = distance.euclidean(start_point, finish_point)
    
    # check distance of each track
    if ride_length < min_ride_length:
        continue
    else:
        all_eastings.append(all_eastings_i[i])
        all_northings.append(all_northings_i[i])
        all_times.append(all_times_i[i])

############# STATISTICS ###########################

print("total time calculated as " + str(total_time) + " seconds")

# find all wave durations
wave_durations = []
for i in range(0, len(all_times), 1):
    wave_durations.append( ((max(all_times[i][0]) - min(all_times[i][0]))/frame_rate)*model_scale_factor_time )

# find average wave duration
average_wave_duration = np.mean(wave_durations)
max_wave_duration = np.max(wave_durations)

print("The average ride duration was " + str(round((average_wave_duration), 2)) + "s.")
print("The maximum ride duration was " + str(round((max_wave_duration), 2)) + "s.")
print("The total time was " + str(round(total_time/60, 2)) + "min.")
print("The ride rate was " + str(round(len(all_times)/(total_time/60), 2)) + "rides/min.")

average_ride_speeds = []
ride_lengths = []
ride_track_angles = []
for i in range(0, len(all_eastings), 1):

    # approximate ride length of waves - simply measure start to finish point
    smooth_easting_i = np.array(all_eastings[i])*model_scale_factor_dist
    smooth_northing_i = np.array(all_northings[i])*model_scale_factor_dist
    start_point = [smooth_easting_i[0][0], smooth_northing_i[0][0]]
    finish_point = [smooth_easting_i[0][-1], smooth_northing_i[0][-1]]
    ride_length = distance.euclidean(start_point, finish_point)
    ride_lengths.append( ride_length )
    
    # average angle of waves
    rise = finish_point[1] - start_point[1]
    run = finish_point[0] - start_point[0]
    ride_angle_deg = find_true_north_dir(rise, run)
    ride_track_angles.append(ride_angle_deg)
    
    # average speed of waves
    average_speed = ride_length/wave_durations[i]
    average_ride_speeds.append(average_speed)
    
print("The average ride length was " + str(round(np.mean(ride_lengths), 2)) + "m.")
print("The maximum ride length was " + str(round(np.max(ride_lengths), 2)) + "m.")
print("The average ride speed was " + str(round(np.mean(average_ride_speeds), 2)) + "m/s.")
print("The maximum ride speed was " + str(round(np.max(average_ride_speeds), 2)) + "m/s.")

# save text file with all stats
with open(results_dir + "/surf_amenity_stats_" + filename + ".txt", 'w') as txt_file:
    txt_file.write("The average ride duration was " + str(round((average_wave_duration), 2)) + "s.\n")
    txt_file.write("The maximum ride duration was " + str(round((max_wave_duration), 2)) + "s.\n")
    txt_file.write("The total time was " + str(round(total_time/60, 2)) + "min.\n")
    txt_file.write("The total number of rides were " + str(len(all_times)) + ".\n")
    txt_file.write("The ride rate was " + str(round(len(all_times)/(total_time/60), 2)) + "rides/min.\n")
    txt_file.write("The average ride length was " + str(round(np.mean(ride_lengths), 2)) + "m.\n")
    txt_file.write("The maximum ride length was " + str(round(np.max(ride_lengths), 2)) + "m.\n")
    txt_file.write("The average ride speed was " + str(round(np.mean(average_ride_speeds), 2)) + "m/s.\n")
    txt_file.write("The maximum ride speed was " + str(round(np.max(average_ride_speeds), 2)) + "m/s.\n")

# initialize file for saving data in
open(results_dir + "/tracked_objects_filt_" + filename + ".csv", "wb")

for i in range(0, len(all_eastings), 1):
    # save all values in csv file [east, north, time]
    data_csv = np.array([all_eastings[i][0], all_northings[i][0], all_times[i][0]])
    # 1 if tracked for more than 'trackable_threshold' frames, 0 if tracked for less
    len_csv = np.array([ride_lengths[i]])
    dir_csv = np.array([ride_track_angles[i]])
    speed_csv = np.array([average_ride_speeds[i]])
    space_csv = np.array([int(1)])
    # write rows of data
    with open(results_dir + "/tracked_objects_filt_" + filename + ".csv", "ab") as csv_file:
        np.savetxt(csv_file, data_csv, delimiter=",", fmt='%f')
        np.savetxt(csv_file, len_csv, delimiter=",", fmt='%f')   
        np.savetxt(csv_file, dir_csv, delimiter=",", fmt='%f')   
        np.savetxt(csv_file, speed_csv, delimiter=",", fmt='%f')   
        np.savetxt(csv_file, space_csv, delimiter=",", fmt='%f')

# show wave trajectories:
fig=plt.figure()
ax=fig.add_subplot(111)
ax.imshow(graph_image, extent=[0, graph_image.shape[1]*easting_scale*model_scale_factor_dist, 
                               graph_image.shape[0]*-northing_scale*model_scale_factor_dist, 0]) # left, right, bottom, top
ax.set_xlabel('Eastings (m)')
ax.set_ylabel('Northings (m)')
ax.set_title('Wave Peel Tracks')
colours = cm.rainbow(np.linspace(0, 1, len(all_eastings)))
for i in range(0, len(all_eastings), 1):
    smooth_easting = np.array(all_eastings[i])*model_scale_factor_dist
    smooth_northing = np.array(all_northings[i])*model_scale_factor_dist
    ax.plot(smooth_easting[0], smooth_northing[0], color=colours[i], lw=0.5)

plt.show()
# save the plot
fig.savefig(results_dir + "/wave_tracks_" + filename + ".png", bbox_inches='tight', dpi=1200)

# show wave trajectories ordered by duration:
fig=plt.figure()
ax0=fig.add_subplot(111)
ax0.imshow(graph_image, extent=[0, graph_image.shape[1]*easting_scale*model_scale_factor_dist, 
                                graph_image.shape[0]*-northing_scale*model_scale_factor_dist, 0]) # left, right, bottom, top
ax0.set_xlabel('Eastings (m)')
ax0.set_ylabel('Northings (m)')
ax0.set_title('Wave Peel Track Durations')
for i in range(0, len(all_eastings), 1):
    smooth_easting = np.array(all_eastings[i])*model_scale_factor_dist
    smooth_northing = np.array(all_northings[i])*model_scale_factor_dist
    
    if wave_durations[i] < 4:
        colour = colours_list[0]
    elif wave_durations[i] < 8:
        colour = colours_list[1]
    elif wave_durations[i] < 12:
        colour = colours_list[2]
    elif wave_durations[i] < 16:
        colour = colours_list[3]
    else:
        colour = colours_list[4]
    
    ax0.plot(smooth_easting[0], smooth_northing[0], color=colour, lw=2, alpha = 0.7)

ax0.legend(custom_lines, legend_list)
plt.show()
# save the plot
fig.savefig(results_dir + "/wave_tracks_dur_" + filename + ".png", bbox_inches='tight', dpi=1200)

# heat map of initial breaking positions
fig=plt.figure()
ax2=fig.add_subplot(111)
ax2.imshow(graph_image, extent=[0, graph_image.shape[1]*easting_scale*model_scale_factor_dist, 
                                graph_image.shape[0]*-northing_scale*model_scale_factor_dist, 0]) # left, right, bottom, top
ax2.set_xlabel('Eastings (m)')
ax2.set_ylabel('Northings (m)')
ax2.set_title('Initial Wave Peel Points')
for i in range(0, len(all_eastings), 1):
    # plot the first points from each wave
    ax2.scatter(all_eastings[i][0][0]*model_scale_factor_dist, 
                all_northings[i][0][0]*model_scale_factor_dist, color='g', alpha = 0.5, s=3)

plt.show()
# save the plot
fig.savefig(results_dir + "/wave_init_brkpnts_" + filename + ".png", bbox_inches='tight', dpi=1200)

# heat map of all breaking positions
fig=plt.figure()
ax3=fig.add_subplot(111)
ax3.imshow(graph_image, extent=[0, graph_image.shape[1]*easting_scale*model_scale_factor_dist, 
                                graph_image.shape[0]*-northing_scale*model_scale_factor_dist, 0]) # left, right, bottom, top
ax3.set_xlabel('Eastings (m)')
ax3.set_ylabel('Northings (m)')
ax3.set_title('All Wave Peel Points')
for i in range(0, len(all_eastings), 1):
    # plot the first points from each wave
    ax3.scatter(all_eastings[i][0]*model_scale_factor_dist, 
                all_northings[i][0]*model_scale_factor_dist, color='r', alpha = 0.4, s=2)

plt.show()
# save the plot
fig.savefig(results_dir + "/wave_all_brkpnts_" + filename + ".png", bbox_inches='tight', dpi=1200)

# histogram of ride lengths
fig=plt.figure()
plt.hist(ride_lengths, density=False, bins=16)
plt.ylabel('Frequency')
plt.xlabel('Ride Length (m)');
plt.show()
fig.savefig(results_dir + "/ride_length_hist_" + filename + ".png", bbox_inches='tight')

# create a ride speed rose petal plot
def get_petal_speed_data(wpt_speeds, wpt_angles, wpt_lengths):
    
    #             N NNE  NE ENE   E ESE  SE SSE   S SSW  SW WSW   W WNW  NW NNW
    hist_bins = [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]
    dir_count =  [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]
    for j in range(0, len(wpt_angles), 1):
        
        direction = wpt_angles[j]

        if direction >= 348.75:
            hist_bins[0] = (hist_bins[0]*dir_count[0] + wpt_speeds[j])/(dir_count[0] + 1)
            dir_count[0] = dir_count[0] + 1
        elif direction < 11.25:
            hist_bins[0] = (hist_bins[0]*dir_count[0] + wpt_speeds[j])/(dir_count[0] + 1)
            dir_count[0] = dir_count[0] + 1
        elif direction >= 11.25 and direction < 33.75:
            hist_bins[1] = (hist_bins[1]*dir_count[1] + wpt_speeds[j])/(dir_count[1] + 1)
            dir_count[1] = dir_count[1] + 1
        elif direction >= 33.75 and direction < 56.25:
            hist_bins[2] = (hist_bins[2]*dir_count[2] + wpt_speeds[j])/(dir_count[2] + 1)
            dir_count[2] = dir_count[2] + 1
        elif direction >= 56.25 and direction < 78.75:
            hist_bins[3] = (hist_bins[3]*dir_count[3] + wpt_speeds[j])/(dir_count[3] + 1)
            dir_count[3] = dir_count[3] + 1
        elif direction >= 78.75 and direction < 101.25:
            hist_bins[4] = (hist_bins[4]*dir_count[4] + wpt_speeds[j])/(dir_count[4] + 1)
            dir_count[4] = dir_count[4] + 1
        elif direction >= 101.25 and direction < 123.75:
            hist_bins[5] = (hist_bins[5]*dir_count[5] + wpt_speeds[j])/(dir_count[5] + 1)
            dir_count[5] = dir_count[5] + 1
        elif direction >= 123.75 and direction < 146.25:
            hist_bins[6] = (hist_bins[6]*dir_count[6] + wpt_speeds[j])/(dir_count[6] + 1)
            dir_count[6] = dir_count[6] + 1
        elif direction >= 146.25 and direction < 168.75:
            hist_bins[7] = (hist_bins[7]*dir_count[7] + wpt_speeds[j])/(dir_count[7] + 1)
            dir_count[7] = dir_count[7] + 1
        elif direction >= 168.75 and direction < 191.25:
            hist_bins[8] = (hist_bins[8]*dir_count[8] + wpt_speeds[j])/(dir_count[8] + 1)
            dir_count[8] = dir_count[8] + 1
        elif direction >= 191.25 and direction < 213.75:
            hist_bins[9] = (hist_bins[9]*dir_count[9] + wpt_speeds[j])/(dir_count[9] + 1)
            dir_count[9] = dir_count[9] + 1
        elif direction >= 213.75 and direction < 236.25:
            hist_bins[10] = (hist_bins[10]*dir_count[10] + wpt_speeds[j])/(dir_count[10] + 1)
            dir_count[10] = dir_count[10] + 1
        elif direction >= 236.25 and direction < 258.75:
            hist_bins[11] = (hist_bins[11]*dir_count[11] + wpt_speeds[j])/(dir_count[11] + 1)
            dir_count[11] = dir_count[11] + 1
        elif direction >= 258.75 and direction < 281.25:
            hist_bins[12] = (hist_bins[12]*dir_count[12] + wpt_speeds[j])/(dir_count[12] + 1)
            dir_count[12] = dir_count[12] + 1
        elif direction >= 281.25 and direction < 303.75:
            hist_bins[13] = (hist_bins[13]*dir_count[13] + wpt_speeds[j])/(dir_count[13] + 1)
            dir_count[13] = dir_count[13] + 1
        elif direction >= 303.75 and direction < 326.25:
            hist_bins[14] = (hist_bins[14]*dir_count[14] + wpt_speeds[j])/(dir_count[14] + 1)
            dir_count[14] = dir_count[14] + 1
        else:
            hist_bins[15] = (hist_bins[15]*dir_count[15] + wpt_speeds[j])/(dir_count[15] + 1)
            dir_count[15] = dir_count[15] + 1

    # Compute pie slices
    N = 16
    theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
    width = 2 * np.pi / N

    return theta, hist_bins, width, dir_count

theta_wpt, speed_bins_wpt, width_wpt, direction_bins_wpt = get_petal_speed_data(average_ride_speeds, 
                                                                                ride_track_angles, ride_lengths)
fig = plt.figure()
ax6 = fig.add_subplot(111, projection='polar')
bars = ax6.bar(theta_wpt, speed_bins_wpt, width=width_wpt, bottom=0.0, alpha=0.5, color = '#f77605')
ax6.set_title("Avg. Ride Speeds For WPT Directions")
ax6.set_theta_zero_location('N')
ax6.set_theta_direction(-1)
ax6.set_ylim([0,15])
plt.show()
fig.savefig(results_dir + "/dir_speed_polar_" + filename + ".png", bbox_inches='tight')

fig = plt.figure()
ax7 = fig.add_subplot(111, projection='polar')
bars = ax7.bar(theta_wpt, direction_bins_wpt, width=width_wpt, bottom=0.0, alpha=0.5, color = 'b')

ax7.set_title("WPT Directions Polar Histogram")
ax7.set_theta_zero_location('N')
ax7.set_theta_direction(-1)
ax7.set_ylim([0,80])
plt.show()
fig.savefig(results_dir + "/dir_hist_" + filename + ".png", bbox_inches='tight')

print("done")